In [1]:
from google.colab import files
uploaded = files.upload()


Saving train.csv to train.csv


In [2]:
from google.colab import files
uploaded = files.upload()


Saving valid.csv to valid.csv


In [3]:
import numpy as np
import random
import pandas as pd
import nltk
import re
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam
!pip install emoji
!pip install arabic-reshaper
nltk.download('punkt')
from nltk import word_tokenize
nltk.download('stopwords')
import re
import string
import emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 11.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
df_train = pd.read_csv('train.csv')
df_train.head()

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...


In [5]:
df_valid = pd.read_csv('valid.csv')
df_valid.head()

,correct,incorrect
0,لا يعتقد البعض بان ضغوطات الغرب علي سوريا هي ش...,لا يعتقد البعض بان ضغوطات الغرب علي سوريا هي ش...
1,انا متاكد ان التعليمات تاتي من الغرب لزعماء ال...,انا متاكد ان التعليمات تاتي من الغرب لزعماء ال...
2,اناشد الاخوه في مصر ارض الكنانه ان يتقوا الله ...,اناشد الاخوه في مصر ارض الكنانه ان يتقوا الله ...
3,علي من ميلانو يقول : ان سوريا بغالبيتها التي ت...,علي من ميلانوا يقول ان سوريا بغالبيته التي تبل...
4,قسما بالله ، انا اعرف اهل حمص ، اليوم شيعنا خي...,قسما بالله انا اعرف اهل حمص اليوم شيعنا خيره ا...


In [6]:
df = pd.concat([df_train, df_valid])
df

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
...,...,...
2288,اعتقد ان اي مسلم من المستحيل ان يفعل هذا ، سوا...,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...
2289,( . . وهو يصلح جدران بيته ويصبغها ، وحينما يست...,( . . وهو يصلح جدران بيته ويصبغها وحينما يستعص...
2290,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...
2291,تصرف طبيعي ، اي انسان في امريكا يتعرض بالنقد ل...,تصرف طبيعي اي انسان في امريكا يتعرض بالنقد للي...


In [7]:
# Function to add a random character to a word in Arabic text
def add_random_arabic_character(word):
    arabic_chars = "ابتثجحخدذرزسشصضطظعغفقكلمنهوي"
    random_char = random.choice(arabic_chars)
    return word + random_char

# Function to remove a random character from a word in Arabic text
def remove_random_arabic_character(word):
    if len(word) <= 1:
        return word  # If the word has only one character, return the same word
    random_index = random.randint(0, len(word) - 1)  # Choose a random index
    return word[:random_index] + word[random_index + 1:]  # Remove the character at the random index



In [8]:
def modify_text(text):
    words = text.split()  # Split the text into words
    if not words:
        return ""  # Return empty string if there are no words
    random_word = random.choice(words)  # Choose a random word
    modify_choice = random.choice(["add", "remove"])  # Choose whether to add or remove a character
    modified_words = []
    for word in words:
        if word == random_word:
            if modify_choice == "add":
                modified_words.append(add_random_arabic_character(word))
            elif modify_choice == "remove":
                modified_words.append(remove_random_arabic_character(word))
        else:
            modified_words.append(word)
    return ' '.join(modified_words)  # Join the modified words back into a single string


In [9]:
df['new_incorrect'] = df['correct'].apply(modify_text)


In [10]:
df.head(10)

,correct,incorrect,new_incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...,النصر ات لا محال ان شاء الله . من يءمن بالله و...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل خص يعتقد ان بشار الاسد سوف يخرج من سوري...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
5,سوريا ام للسوريين ، او لبشار وعصابته ؟ لن تنته...,سوريا اما للسوريين او لبشار وعصابته لن تنتهي ه...,سوريا ام للسوريين ، او لبشار وعصابته ؟ لن تنته...
6,التاريخ يعيد نفسه . الاخوان المجرمون يقتلون وي...,التاريخ يعيد نفسه . الاخوان المجرمون يقتلون و ...,التاريخ يعيد نفسه . الاخوان المجرمون يقتلون وي...
7,عار عليكم يا امه الاسلام لعدم مساعدتكم لنا ، ن...,عار عليكم يا امه الاسلام لعدم مساعدتكم لنا نحن...,عار عليكم يا امه الاسلام لعدم مساعدتكم لنا ، ن...
8,تنويه : عقرب هي قريه تقع في سهل الحوله ، وتتبع...,تنويه : عقرب هي قريه تقع في سهل الحوله وتتبع ا...,تنويه : عقرب هي قريه تقع في سهل الحوله ، وتتبع...
9,ضحكوا علينا بحرب العراق عام 1990 ، وعام 2003 ،...,ضحكو علينا بحرب العراق عام ١٩٩٠ وعام ٢٠٠٣ وحرب...,ضحكوا علينا بحرب العراق عام 1990 ، وعام 2003 ،...


In [11]:
df=df[["correct","new_incorrect"]]
df

,correct,new_incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال ان شاء الله . من يءمن بالله و...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل خص يعتقد ان بشار الاسد سوف يخرج من سوري...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
...,...,...
2288,اعتقد ان اي مسلم من المستحيل ان يفعل هذا ، سوا...,اعتقد ان اي مسلم من المستحيل ان يفعل هذا ، سوا...
2289,( . . وهو يصلح جدران بيته ويصبغها ، وحينما يست...,( . . وهو يصلح جدران بيته ويصبغها ، وحينما يست...
2290,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...,انا استغرب من ينكر علي الجزيره ويصفها باثاره ا...
2291,تصرف طبيعي ، اي انسان في امريكا يتعرض بالنقد ل...,تصرف طبيعي ، اي انسان في امريكاب يتعرض بالنقد ...


In [ ]:
arabicPunctuations = [".","#","$","//","?","=","'","_","-","'",";","\\","`","؛","<",">","(",")","*","&","^","%","]","[",",","ـ","،","/",":","؟",".","'","{","}","~","|","!","”","…","“","–"] # defining customized punctuation marks


def removingPunctuation(text):
  cleanTweet = ''
  for i in text:
    if i not in arabicPunctuations:
      cleanTweet = cleanTweet + '' + i
  return cleanTweet

df['correct'] = df['correct'].apply(lambda x: removingPunctuation(x))
df['new_incorrect'] = df['new_incorrect'].apply(lambda x: removingPunctuation(x))


df.sample(5)

<ipython-input-12-512aa89f238d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['correct'] = df['correct'].apply(lambda x: removingPunctuation(x))
<ipython-input-12-512aa89f238d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_incorrect'] = df['new_incorrect'].apply(lambda x: removingPunctuation(x))


,correct,new_incorrect
11996,الي اعضاء حزب عبده عاوزين يعترضوا نقول لهم ...,الي اعضاء زب عبده عاوزين يعترضوا نقول لهم ت...
9961,لماذا لا تتحرك طاءرات قطر مثلما تحركت لقصف ليب...,لماذا لا تتحرك طاءرات قطر مثلما تحركت لقصف ليب...
7611,الموت للقتله المندسين عن الاسلام ما هكذا الذي...,الموت للقتله المندسين عن الاسلامض ما هكذا الذ...
16075,غريب وعجيب امر امريكا والدول الاوروبيه فلا ي...,غريبخ وعجيب امر امريكا والدول الاوروبيه فلا ...
900,هذا غير صحيح المقال يفقد كل معايير المصداقيه ...,هذا غير صحيح المقال يفقد كل معايير المصداقيه ...


In [13]:
def clean(text):
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text)
    # Remove numbers
    text = re.sub("\d+", " ", text)
    return text

df['correct'] = df['correct'].apply(lambda x:clean(x))

df['new_incorrect'] = df['new_incorrect'].apply(lambda x:clean(x))

<ipython-input-13-9f8dc86271b0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['correct'] = df['correct'].apply(lambda x:clean(x))
<ipython-input-13-9f8dc86271b0>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_incorrect'] = df['new_incorrect'].apply(lambda x:clean(x))


In [14]:
df

,correct,new_incorrect
0,سبحان الله الحكام العرب سيموت علي الكرسي ليظهر...,سبحان الله الحكام العرب سيموت علي الكرسي ليظهر...
1,النصر ات لا محال ان شاء الله من يءمن بالله وال...,النصر ات لا محال ان شاء الله من يءمن بالله وال...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل خص يعتقد ان بشار الاسد سوف يخرج من سوري...
3,الاسد وعصابته لحد الان غير مستوعبين انه رح ينق...,الاسد وعصابته لحد الان غير مستوعبين انه رح ينق...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
...,...,...
2288,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...
2289,وهو يصلح جدران بيته ويصبغها وحينما يستعصي علي...,وهو يصلح جدران بيته ويصبغها وحينما يستعصي علي...
2290,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...,انا استغرب من ينكر علي الجزيره ويصفها باثاره ا...
2291,تصرف طبيعي اي انسان في امريكا يتعرض بالنقد للي...,تصرف طبيعي اي انسان في امريكاب يتعرض بالنقد لل...


In [15]:
df.sample(10)

,correct,new_incorrect
4448,المعارضه دعت الي جمعه الحوار وسعت لتخريب البلد...,المعارضه دعت الي جمعه الحوار وسعت لتخريب البلد...
8029,كل اخواننا العرب المعلقين اقول لكم كلمه نحن شع...,كل اخواننا العرب المعلقين اقول لكم كلمه نحن شع...
1681,واين ملوك وامراء العرب المصلون ليلا ونهارا قصو...,واين ملوك وامراء العرب ح د المصلون ليلا ونهارا...
8581,الي الذين يقدحون كلام الاستاذ بعاطفه الدفاع بد...,الي الذين يقدحون كلام الاستاذ بعاطفه الدفاع بد...
16110,وماذا صنعت انت يا من تدعو الي النفير في سبيل ا...,وماذا صنعت انت يا من تدعو الي النفير في سبيل ا...
1708,الي كل القراء اقسم بالله ان اقول الحق انا تاجر...,الي كل القراء اقسم بالله ان اقول الحق انا تاجر...
5483,كل ما تذكره كل الشعب العربي مءيد له فهل تريد ...,كل ما تذكره كل الشعب العربي مءيد له فهل تريد ...
15663,عن ايه منظمه حقوقيه تتكلمون اين كنت يا منظمه ع...,عن ايه مظمه حقوقيه تتكلمون اين كنت يا مظمه عند...
9249,يجب ان يرسل للثوار دعما للثوره والثوار وليدهم ...,يجب ان يرسل للثوار دعما للثوره والثوار وليدهم ...
10744,حرام عليكم يا جزيره انتم بتحريضكم من يقتل السو...,حرام عليكم يا جزيره انتم بتحريضكم من يقتل السو...


In [16]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
tokenizer.fit_on_texts(df['correct'])
correct_sequences = tokenizer.texts_to_sequences(df['correct'])
incorrect_sequences = tokenizer.texts_to_sequences(df['new_incorrect'])

# Set the maximum sequence length

max_sequence_length = max(max(len(seq) for seq in correct_sequences), max(len(seq) for seq in incorrect_sequences))

# Pad sequences to ensure uniform length
correct_sequences = pad_sequences(correct_sequences, maxlen=max_sequence_length, padding='post')
incorrect_sequences = pad_sequences(incorrect_sequences, maxlen=max_sequence_length, padding='post')


In [17]:
len(tokenizer.word_index)

100874

In [18]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(incorrect_sequences, correct_sequences, test_size=0.2, random_state=42)



In [19]:
# Model architecture
input_seq = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_seq)
encoder = LSTM(128, return_sequences=True)(embedding_layer)
decoder = Dense(len(tokenizer.word_index) + 1, activation='softmax')(encoder)

# Define the model
model = Model(input_seq, decoder)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()





Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 394)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 394, 128)       │    12,912,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 394, 128)       │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 394, 100875)    │    13,012,875 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,056,459 (99.40 MB)

 Trainable params: 26,056,459 (99.40 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=5)


Epoch 1/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 636s 612ms/step - accuracy: 0.8676 - loss: 2.4138 - val_accuracy: 0.8770 - val_loss: 1.0995
Epoch 2/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 685s 617ms/step - accuracy: 0.8805 - loss: 1.0691 - val_accuracy: 0.8925 - val_loss: 0.9964
Epoch 3/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 684s 619ms/step - accuracy: 0.8969 - loss: 0.9287 - val_accuracy: 0.9065 - val_loss: 0.8836
Epoch 4/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 692s 670ms/step - accuracy: 0.9110 - loss: 0.7826 - val_accuracy: 0.9161 - val_loss: 0.7932
Epoch 5/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 742s 671ms/step - accuracy: 0.9243 - loss: 0.6449 - val_accuracy: 0.9228 - val_loss: 0.7335


In [21]:
# تقييم النموذج على مجموعة التحقق
loss, accuracy = model.evaluate(X_val, y_val, verbose=1, batch_size=16)



259/259 ━━━━━━━━━━━━━━━━━━━━ 89s 343ms/step - accuracy: 0.9224 - loss: 0.7357
